<center>
<table>
  <tr>
    <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/nasa-logo.svg" width="100"/> </td>
     <td><img src="https://portal.nccs.nasa.gov/datashare/astg/training/python/logos/ASTG_logo.png?raw=true" width="80"/> </td>
     <td> <img src="https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png" width="130"/> </td>
    </tr>
</table>
</center>

        
<center>
<h2><font color= "blue" size="+3">PyCon 2024 Tutorial</font></h2>
</center>

---

<center>
    <h3>Python Workflows to Extract and Plot Satellite Data Products along Tracks</h3>
    <h2><font color="red" size="+3">2024 Solar Eclipse - Option 1</font></h2>
</center>

## <font color="blue">Objective</font>

We want to:

- Plot the path of the April 8, 2024 solar eclipse.
- Identify the cities that will have full view of the total eclipse.

Part of this work was taken from: [Plan Your Next Eclipse Viewing](https://github.com/christyheaton/PlanYourNextEclipseViewing/blob/master/Notebook/Presentation.ipynb) by Christy Heaton

## <font color="blue">Import necessary modules</font>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import datetime as dt
from pathlib import Path

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
from shapely import geometry as shpgeom
from shapely import wkt as shpwkt

In [ ]:
import movingpandas as mpd

In [ ]:
import holoviews as hv
import hvplot.pandas 

In [ ]:
figsize = (17, 11)

## <font color="blue">Read the CSV file with Pandas</font>

We used the web document:

[Predicting solar eclipses with Python](https://erikbern.com/2024/04/07/predicting-solar-eclipses-with-python.html) by Erik Bernhardsson

to create a CSV file that has three columns:
- `t`: for the date/time as strings
- `latitude` as floats
- `longitude` as floats

We want here to use Pandas to read the CSV file, and:

- Transform the `t` values into datetime objects.
- Rename the columns.
- Create a new `geometry` columns

In [ ]:
#data_dir = "/Users/jkouatch/myTasks/PythonTraining/ASTG606/Materials/sat_data/Eclipse_Data/"
data_dir = "/tljh-data/sat_data/Eclipse_Data"

In [ ]:
file_name = Path(data_dir) / "eclipse_path_8April2024.csv"

#### <font color="green">Read the file to create a Pandas DataFrame</font>

Do no forget to handle the date/time.

In [ ]:
dateparse = lambda x: ...
df_eclipse = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
dateparse = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df_eclipse = pd.read_csv(file_name,
                parse_dates={'datetime': [0,]}, 
                date_parser=dateparse)
```
</details>

#### <font color="green">Rename the `datetime` column to `t`</font>

In [ ]:
df_eclipse.

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
df_eclipse.rename(columns = {'datetime': 't'}, inplace=True)
```
</details>

#### <font color="green">Add a `geometry` column</font>

In [ ]:
df_eclipse['geometry'] = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
df_eclipse['geometry'] = [shpgeom.Point(xy) for xy in zip(df_eclipse['longitude'], df_eclipse['latitude'])] 
```
</details>

## <font color="blue">Manipulation with GeoPandas</font>

#### <font color="green">Use GeoPandas to create the GeoDataFrame</font>

In [1]:
gdf_eclipse = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
gdf_eclipse = gpd.GeoDataFrame(df_eclipse, geometry="geometry") 
```
</details>

#### Interactive plot of the path of the eclipse

In [ ]:
gdf_eclipse.hvplot(tiles='EsriTerrain', coastline=True, 
           hover_cols=["t"])

#### Create a buffer along the path
- The buffer will correspond to the area along the path where we view the total elcipse.

In [ ]:
size_buffer = 1.5

In [ ]:
gdf_eclipse_buffer = gpd.GeoDataFrame(geometry=gdf_eclipse.buffer(size_buffer))
gdf_eclipse_buffer

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
gdf_eclipse.plot(ax=ax, color="blue", alpha=1.0)
gdf_eclipse_buffer.plot(ax=ax, color="pink", alpha=0.2);

## <font color="blue">Manipulation with MovingPandas</font>

#### <font color="green">Create the MovingPandas trajectory</font>

In [ ]:
mdf_eclipse = ...

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
mdf_eclipse = mpd.Trajectory(df_eclipse, traj_id=1,
                      x = "longitude", y="latitude", t="t")
```
</details>

#### Basic plot of the trajectory and the buffer

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,10))
gdf_eclipse.plot(ax=ax, color="pink", )
mdf_eclipse.plot(ax=ax);

#### Interactive plot of the trajectory

In [ ]:
mdf_eclipse.hvplot(line_width=0.9)

## <font color="blue">GeoDataFrame for all the countries of the world</font>

- We use the Naturel Earth database of the polygon geometry of each country of world

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world

In [ ]:
world.plot(color='white', linewidth=0.5, edgecolor='black', figsize=figsize);

## <font color="blue">GeoDataFrame for all the major cities of the world</font>

In [ ]:
url_cities = "https://github.com/nvkelso/natural-earth-vector/raw/master/10m_cultural/ne_10m_populated_places.shp"
world_cities = gpd.read_file(url_cities)
world_cities.head()

In [ ]:
base = world.plot(color='white', linewidth=0.5, edgecolor='black', figsize=figsize)
world_cities.plot(ax=base, color='orange', markersize=3)
base.set_axis_off() 

## <font color="blue">Identify the cities that will have full view of the solar eclipse</font>

In [ ]:
cities_eclipse = gpd.sjoin(world_cities, gdf_eclipse_buffer, 
                           how='inner', op='intersects')
cities_eclipse

#### Plot the cities that will have full view of the solar eclipse

In [ ]:
base = world.plot(color='white', linewidth=0.5, edgecolor='black', figsize=figsize)
cities_eclipse.plot(ax=base, color='orange', markersize=3)
base.set_axis_off() 

#### Use the `world` GeoDataFrame to zoom in on North America

In [ ]:
north_america = world[world['continent']=='North America']

#### Plot the cities that will have full view of the solar eclipse

In [ ]:
base = north_america.plot(color='white', linewidth=0.5, edgecolor='black', figsize=figsize)
cities_eclipse.plot(ax=base, color='orange', markersize=3)
base.set_axis_off() 

#### <font color="green">Plot the solar eclipse path (buffer) and the cities</font>

Include the area along the path where the total eclipse will be seen.

In [ ]:
base = north_america.plot(color='white', linewidth=0.5, edgecolor='black', figsize=figsize)
cities_eclipse.plot(ax=base, color='orange', markersize=3)
base.set_axis_off() 

<details><summary><b><font color="purple">Click here to access the solution</font></b></summary>
<p>

```python
base = north_america.plot(color='white', linewidth=0.5, edgecolor='black', figsize=figsize)
cities_eclipse.plot(ax=base, color='orange', markersize=3)
gdf_eclipse_buffer.plot(ax=base, color="black", alpha=0.03);
base.set_axis_off() 
```
</details>